INPUTS
- `sset.PATH_CIAM_SITES_WITHISO`

OUTPUTS
- `sset.PATH_CIAM_SITES_VORONOI_BY_ISO`

In [1]:
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import box

from sliiders import spatial as pv
from sliiders import settings as sset

pv.filter_spatial_warnings()

In [2]:
stations = gpd.read_parquet(sset.PATH_CIAM_SITES_WITHISO)

# Rename columns for compatibility with Voronoi functions
stations = stations.rename(columns={"lat": "y", "lon": "x", "station_id": "UID"})

Make sure none of the stations with too few points to calculate SphericalVoronoi are anywhere near the poles, so we can introduce the poles as extra points

In [3]:
iso_count = pd.DataFrame(stations.groupby("ISO")["ISO"].count()).rename(
    columns={"ISO": "count"}
)
stations = stations.join(iso_count, on="ISO")

assert stations[stations["count"] <= 3]["y"].max() < 60
assert stations[stations["count"] <= 3]["y"].min() > -60

### Define single-country Voronoi function

In [4]:
def append_extra_pts(sites):
    """Define three extra points at the pole farthest from any station"""
    ymax = sites["y"].max()
    ymin = sites["y"].min()
    nsign = -1 if np.abs(ymax) > np.abs(ymin) else 1

    extra_pts = pd.DataFrame(
        {
            "UID": ["placeholder1", "placeholder2", "placeholder3"],
            "y": [90 * nsign, 89 * nsign, 89 * nsign],
            "x": [0, 0, 180],
        }
    )

    return pd.concat([iso_stations, extra_pts])


def get_voronoi_from_sites(sites):
    """Get the Voronoi diagram corresponding to the stations defined by `iso_stations`"""
    if sites.shape[0] == 1:
        vor_gdf = sites[["UID", "geometry"]].copy()
        vor_gdf["geometry"] = box(-180, -90, 180, 90)
    else:
        if sites.shape[0] <= 3:
            sites = append_extra_pts(sites)
        vor_gdf = pv.get_spherical_voronoi_gdf(sites, show_bar=False)

    return vor_gdf

### Iterate through each country, add each Voronoi gdf to `vors`

In [5]:
all_isos = stations["ISO"].unique()

all_isos.sort()

vors = []
for iso in all_isos:
    print(iso, end=" ")
    iso_stations = stations[stations["ISO"] == iso].copy()
    vors.append(get_voronoi_from_sites(iso_stations))

ABW AGO AIA ALA ALB ARE ARG ASM ATF ATG AUS BEL BEN BES BGD BGR BHR BHS BIH BLM BLZ BMU BRA BRB BRN BVT CAN CCK CHL CHN CIV CMR COD COG COK COL COM CPV CRI CUB CUW CXR CYM CYP DEU DJI DMA DNK DOM DZA ECU EGY ERI ESH ESP EST FIN FJI FLK FRA FRO FSM GAB GBR GEO GGY GHA GIB GIN GLP GMB GNB GNQ GRC GRD GRL GTM GUF GUM GUY HKG HMD HND HRV HTI IDN IMN IND IOT IRL IRN IRQ ISL ISR ITA JAM JEY JOR JPN KEN KHM KIR KNA KOR KWT LBN LBR LBY LCA LKA LTU LVA MAF MAR MCO MDG MDV MEX MHL MLT MMR MNE MNP MOZ MRT MSR MTQ MUS MYS MYT NAM NCL NFK NGA NIC NIU NLD NOR NRU NZL OMN PAK PAN PCN PER PHL PLW PNG POL PRI PRK PRT PSE PYF QAT REU ROU RUS SAU SDN SEN SGP SGS SHN SJM SLB SLE SLV SOM SPM STP SUR SVN SWE SXM SYC SYR TCA TGO THA TKL TLS TON TTO TUN TUR TUV TWN TZA UKR UMI URY USA VCT VEN VGB VIR VNM VUT WLF WSM XAD XCL XNC XSP YEM ZAF 

### Combine all Voronoi diagrams into one GeoDataFrame (results overlap)

In [6]:
vor_gdf = pd.concat(vors, ignore_index=True)

#### Retrieve ISO from station ID

In [7]:
vor_gdf = vor_gdf.join(stations[["UID", "ISO"]].set_index("UID"), on="UID")

In [8]:
vor_gdf = vor_gdf.rename(columns={"UID": "station_id"})

#### Check that ISOs match

In [9]:
assert set(vor_gdf.loc[vor_gdf["ISO"].isnull(), "station_id"].unique()) - set(
    ["placeholder1", "placeholder2", "placeholder3"]
) == set([])

In [10]:
vor_gdf = vor_gdf[vor_gdf["ISO"].notnull()].copy()

In [11]:
vor_gdf["geometry"] = vor_gdf["geometry"].apply(pv.grab_polygons)

In [12]:
vor_gdf.to_parquet(sset.PATH_CIAM_SITES_VORONOI_BY_ISO, index=False)